In [5]:
from haystack.nodes import PromptNode, WebSearch, PromptTemplate
from haystack.agents import Agent, Tool

import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
"""
Model instantiation with a PromptNode class.

"""

model_api_key = os.getenv("HF_API_KEY")

prompt_node = PromptNode(
    model_name_or_path = "HuggingFaceH4/zephyr-7b-beta",
    api_key = model_api_key,
    max_length = 256,
)

In [7]:
"""
WebSearch()

:param api_key: API key for the search engine provider.
:param search_engine_provider: Name of the search engine provider class. The options are "SerperDev" (default), "SearchApi", "SerpAPI", "BingAPI" or "GoogleAPI"
:param search_engine_kwargs: Additional parameters to pass to the search engine provider.
:param top_search_results: Number of top search results to be retrieved.
:param top_k: Top k documents to be returned by the retriever.
:param mode: Whether to return snippets, raw documents, or preprocessed documents. Snippets are the default.
:param preprocessor: Optional PreProcessor to be used to split documents into paragraphs. If not provided, the default PreProcessor is used.
:param cache_document_store: DocumentStore to be used to cache search results.
:param cache_index: Index name to be used to cache search results.
:param cache_headers: Headers to be used to cache search results.
:param cache_time: Time in seconds to cache search results. Defaults to 24 hours.
:param allowed_domains: List of domains to restrict the search to. If not provided, the search is unrestricted.
:param link_content_fetcher: LinkContentFetcher to be used to fetch the content from the links. If not provided,
the default LinkContentFetcher is used.

"""

search_api_key = os.getenv("SERPAPI_API_KEY")

search = WebSearch(
                api_key = search_api_key,
                search_engine_provider = "SerpAPI",
            )


agent = Agent(
            prompt_node = prompt_node
        )

agent.add_tool(
	Tool(
		name = "WebSearch",
		pipeline_or_node = search,
		description = "Useful when you need to find answers to questions on the internet."
    ))

d:\Projects\agent-prototyping\.venv\Lib\site-packages\haystack\nodes\prompt\prompt_template.py:444: UserWarning: You're using a legacy prompt template 'zero-shot-react', we strongly suggest you use prompts from the official Haystack PromptHub: https://prompthub.deepset.ai/
  warnings.warn(


In [13]:
import pprint

pprint.pprint(vars(agent))
pprint.pprint(agent.has_tool("WebSearch"))

{'callback_manager': <events.events.Events object at 0x2e724337c10>,
 'final_answer_pattern': 'Final Answer\\s*:\\s*(.*)',
 'hash': 'd41d8cd98f00b204e9800998ecf8427e',
 'last_hash': None,
 'max_steps': 8,
 'memory': <haystack.agents.memory.no_memory.NoMemory object at 0x000002E724335950>,
 'prompt_node': <haystack.nodes.prompt.prompt_node.PromptNode object at 0x000002E724336250>,
 'prompt_parameters_resolver': <function react_parameter_resolver at 0x000002E7242FE0C0>,
 'prompt_template': PromptTemplate(name=zero-shot-react, prompt_text=You are a helpful and knowledgeable agent. To achieve your goal of answering complex questions correctly, you have access to the following tools:{new_line}{new_line}{tool_names_with_descriptions}{new_line}{new_line}To answer questions, you'll need to go through multiple steps involving step-by-step thinking and selecting appropriate tools and their inputs; tools will respond with observations. When you are ready for a final answer, respond with the `Fina